In [1]:
import pandas as pd
pd.set_option('max_columns', None)
hotels = pd.read_csv("input/goibibo_com-travel_sample.csv", encoding='latin1')
hotels.tail(10)

,additional_info,address,area,city,country,crawl_date,guest_recommendation,hotel_brand,hotel_category,hotel_description,hotel_facilities,hotel_star_rating,image_count,latitude,locality,longitude,pageurl,point_of_interest,property_id,property_name,property_type,province,qts,query_time_stamp,review_count_by_category,room_area,room_count,room_facilities,room_type,similar_hotel,site_review_count,site_review_rating,site_stay_review_rating,sitename,state,uniq_id
3990,NaN,"Padora enclave patnitop,near nag Temple",Padora Enclave,Patnitop,India,2016-06-26,50.0,null,regular,The standard check-in time is 12:00 PM and the...,Front Desk|24 Hour Front Desk|RO Water Purific...,0,3,33.084705,Padora Enclave,75.335387,http://www.goibibo.com/hotels/detail/Patnitop/...,The Naag (cobra) temple|Madhatop|Kud|Padora En...,HTLGBO1000028680,Hotel Shiva Cottage,Guest House,Patnitop,NaN,2016-06-26 05:31:53 +0000,positive reviews::1|critical reviews::1|review...,NaN,6,Room Service |Ceiling Fan| Hot / Cold Running ...,Double Bedroom,https://www.goibibo.com/hotels/wood-castle-res...,2.0,3.0,Service Quality::3.0|Amenities::3.0|Food and D...,goibibo,Jammu and Kashmir,b7711fc445c54041b18a42367bb8903d
3991,Room Service|Internet Access|Restaurant,"Opp City Power House Jaipur Road, Ajmer",Jaipur Road,Ajmer,India,2016-06-26,74.0,null,regular,The standard check-in time is 12:00 PM and the...,Business Center |Currency Exchange|Doctor on C...,3,22,26.464529,Jaipur Road,74.635719,http://www.goibibo.com/hotels/detail/Ajmer/hn/...,NaN,HTLGBO1000001495,Hotel Siddharth,Hotel,Ajmer,NaN,2016-06-26 05:31:53 +0000,positive reviews::152|critical reviews::61|rev...,13 X 13 Sq.ft.,20,Air Conditioning |Alarm Clock|Newspapers| Hot ...,Deluxe AC Room,https://www.goibibo.com/hotels/kanak-sagar-hot...,213.0,3.6,Service Quality::3.5|Amenities::3.5|Food and D...,goibibo,Rajasthan,b24d7f7b48305a536f0f625df03b48fd
3992,Room Service|Internet Access|Restaurant|Free I...,"sarvaritu vilas , inside udaipole , opp sagasj...",Udaipole,Udaipur,India,2016-06-26,81.0,null,gostays,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Doctor ...,2,7,24.575598,Udaipole,73.696959,http://www.goibibo.com/hotels/detail/Udaipur/h...,Jagdish Temple|City Palace|Udaipur Bus Stand|U...,HTLGBO1000012898,Hotel Shubh Laxmi,Hotel,Udaipur,NaN,2016-06-26 05:31:53 +0000,positive reviews::122|critical reviews::13|rev...,10 X 13 Sq.ft.,44,Room Service |Basic Bathroom Amenities|Cable /...,Deluxe Non A/c Room,https://www.goibibo.com/hotels/north-star-hote...,135.0,4.2,Service Quality::4.2|Amenities::4.0|Food and D...,goibibo,Rajasthan,32cccd16945dbe2ca9f2ea7e59827401
3993,Room Service|Internet Access|Restaurant|Free I...,"Near Yasholaxmi Nagar, Kolhapur Rd,",Yasholaxmi Nagar,Ichalkaranji,India,2016-06-26,100.0,null,regular,The standard check-in time is 10:00 AM and the...,Bar / Lounge |Doctor on Call|Dry Cleaning|Inte...,2,6,16.690042,Yasholaxmi Nagar,74.456394,http://www.goibibo.com/hotels/detail/Ichalkara...,Yasholaxmi Nagar,HTLGBO1000083318,Hotel Signature,Hotel,Ichalkaranji,NaN,2016-06-26 05:31:53 +0000,positive reviews::1|critical reviews::0|review...,NaN,8,Room Service |Air Conditioning |Alarm Clock|Ca...,Suite Room,https://www.goibibo.com/hotels/grand-fortune-h...,1.0,5.0,Service Quality::5.0|Amenities::5.0|Food and D...,goibibo,Maharashtra,4ddfeeb91363fe5692682ef91a9c45cd
3994,Room Service|Restaurant,"Near Railway Station , Kota Jn",Station Road Area,Kota,India,2016-06-26,72.0,null,regular,The standard check-in time is 12:00 PM and the...,Bar / Lounge |Doctor on Call|Laundry Service A...,2,10,25.222694,Station Road Area,75.878170,http://www.goibibo.com/hotels/detail/Kota/hn/5...,Kota Airport|Station Road Area,HTLGBO1000009343,Hotel Shree Anand Restaurant &amp; Bar,Hotel,Kota,NaN,2016-06-26 05:31:53 +0000,positive reviews::104|critical reviews::48|rev...,NaN,16,Room Service |Air Conditioning |Basic Bathroom...,AC ROOM,https://www.goibibo.com/hotels/venus-plaza-hot...,152.0,3.4,S

In [2]:
import numpy as np

# Helper functions for encoding.
def split_piped_list(srs, col):
    try:
        ret = [r.split("::")[-1] for r in srs[col].split("|")]
        ret = [float(r) if len(r) > 0 else np.nan for r in ret]
        return ret
    except AttributeError:
        return np.nan

# Encode the aggregated 5-star review categories into columns.
ratings = pd.DataFrame(data=hotels.apply(
                                lambda h: split_piped_list(h, 'site_stay_review_rating'), 
                                                           axis='columns').tolist(),
                       columns=['service_quality_rating', 'amenities_rating', 
                                'food_and_drinks_rating', 'value_for_money_rating', 
                                'location_rating', 'cleanliness_rating'])
hotels = hotels.join(ratings)

# Encode the reviews column into separate columns.
review_counts = pd.DataFrame(
    data=(
        hotels
            .apply(lambda h: split_piped_list(h, 'review_count_by_category'), axis='columns')
            .map(lambda v: [0, 0, 0] if isinstance(v, float) else v).tolist()
    ),
    columns=['positive_reviews_total', 'critical_reviews_total', 'reviews_with_images_total']
)
hotels = hotels.join(review_counts)

hotels = hotels.drop(['country', 'sitename', 'review_count_by_category', 
                      'site_stay_review_rating'], axis='columns')
hotels = hotels.drop(hotels['room_count'].argmax())  # bad entry